In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import glob
import os.path

# Loads the JPEG image at the given path
# Decodes the JPEG image to a uint8 W X H X 3 tensor
# Resizes the image to 224 x 224 x 3 tensor
# Returns the pre processed image as 224 x 224 x 3 tensor

def load_img(path):
 # Reads the image file and returns data type of string
 img = tf.io.read_file(path)
 # Decodes the image to W x H x 3 shape tensor with type of uint8
 img = tf.io.decode_jpeg(img, channels=3)
 # Resizes the image to 224 x 224 x 3 shape tensor
 img = tf.image.resize_with_pad(img, 224, 224)
 # Converts the data type of uint8 to float32 by adding a new axis
 # img becomes 1 x 224 x 224 x 3 tensor with data type of float32
 # This is required for the mobilenet model we are using
 img = tf.image.convert_image_dtype(img,tf.float32)[tf.newaxis, ...]
 
 return img

# Loads the mobilenet model in TF.HUB
# Makes an inference for all images stored in a local folder
# Saves each of the feature vectors in a file

def get_image_feature_vectors():
 
 # Definition of module with using tfhub.dev
 module_handle = "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4"
 # Loads the module
 module = hub.load(module_handle)
 # Loops through all images in a local folder
 for filename in glob.glob('pic/*.jpg'):
 
  print(filename)
  # Loads and pre-process the image
  img = load_img(filename)
  # Calculate the image feature vector of the img
  features = module(img)
  # Remove single-dimensional entries from the 'features' array  
  feature_set = np.squeeze(features)
  print(feature_set.shape)
  # Saves the image feature vectors into a file for later use
  outfile_name = os.path.basename(filename) + ".npz"
  out_path = os.path.join('output/', outfile_name)
  # Saves the 'feature_set' to a text file
  np.savetxt(out_path, feature_set, delimiter=',')

get_image_feature_vectors()


pic\pie_chair_01.jpg
(1792,)


In [2]:
module_handle = "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4"
module = hub.load(module_handle)

In [3]:
img = load_img('pic/pie_chair_01.jpg')
features = module(img)
target_image = np.squeeze(features)
target_image

array([0.02288028, 1.3349519 , 0.        , ..., 0.01153526, 0.        ,
       0.90349376], dtype=float32)

In [4]:
img = load_img('datas/bed_13110.png')
features = module(img)
input_image = np.squeeze(features)
input_image

array([0.18597327, 0.4929459 , 0.        , ..., 0.31545338, 0.        ,
       0.        ], dtype=float32)

In [5]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return "코사인 유사도 : {}".format(dot(A, B)/(norm(A)*norm(B)))
cos_sim(target_image, input_image)

'코사인 유사도 : 0.6705036759376526'